In [ ]:
import os
import sys
from pathlib import Path

import yaml
import numpy as np
import pydicom
pydicom.config.settings.reading_validation_mode = pydicom.config.IGNORE

import SimpleITK as sitk
from box import ConfigBox

PROJ_ROOT = Path.cwd().parent
if str(PROJ_ROOT) not in sys.path:
    sys.path.append(str(PROJ_ROOT))

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from ipywidgets import interact

from dataset.common import load_all_patient_series, group_dicom_by_series, get_patient_series_map

In [ ]:
yaml_path = PROJ_ROOT / 'config' / 'config.yaml'

with open(yaml_path, 'r', encoding='utf-8') as f:
    cfg = ConfigBox(yaml.safe_load(f))

# Make Patient Image-XML Matching Table

## Dicom Files

### Load Folders across all patients

In [ ]:
patient_series_map = get_patient_series_map(os.path.join(cfg.paths.raw_data, 'patient'))
patient_idx = list(patient_series_map.keys())

In [ ]:
final_patient_data = {}

for p_id, folders in patient_series_map.items():
    final_patient_data[p_id] = {}
    
    for folder in folders:
        grouped_series = group_dicom_by_series(folder, p_id)
        final_patient_data[p_id].update(grouped_series)

In [ ]:
series_folders = final_patient_data['146']
len(series_folders)

In [ ]:
s_folder

In [ ]:
for i, s_folder in enumerate(series_folders):
    files = list(s_folder.glob("*.dcm"))
    if not files: continue
    
    slices = [pydicom.dcmread(f) for f in files]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    
    # 세션 구분용 이름 만들기 (SeriesNumber 태그 활용)
    series_num = getattr(slices[0], 'SeriesNumber', i)
    all_series_data[f"Series_{series_num}"] = slices

### Example - CT Images

In [ ]:
temp = load_all_patient_series('146', final_patient_data)

In [ ]:
def browse_images(idx):
    plt.figure(figsize=(5, 5))
    plt.imshow(temp[idx].pixel_array, cmap='gray')
    plt.title(f"Instance: {temp[idx].InstanceNumber} || Total: {len(temp)}")
    plt.axis('off')
    plt.show()

In [ ]:
interact(browse_images, idx=(0, len(temp)-1))

## XML Files

In [ ]:
xml_path = os.path.join(cfg.paths.raw_data, 'calcium_xml')
temp_xml_path = os.path.join(xml_path, '1.xml')
temp_xml = load_single_xml(temp_xml_path)

In [ ]:
temp_xml[0]